### Open a browser and set the cookies from a JSON file

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time,json, random

def setCookiesFromJson():
    with open('cookies.json', 'r', newline='') as inputdata:
        cookies = json.load(inputdata)
    for cookie in cookies: #works only after driver.get
        driver.add_cookie(cookie)
    driver.refresh() # to load cookies

service = Service(executable_path="chromedriver.exe")
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging']) #disable error logging
driver = webdriver.Chrome(service=service, options=chrome_options)

base_url = "https://theprotocol.it/filtry/python;t"
# driver.get("https://theprotocol.it/filtry/python;t/trainee,assistant,junior;p/zdalna;rw/praca/data-scientist-deep-learning-warszawa,oferta,8e0c0000-a5a4-c69a-3164-08dcbb7766ef?s=-1499548823&searchId=20a1f390-66d0-11ef-8a60-57a9d1202edd")
driver.get(base_url)
setCookiesFromJson()

1


### Fetch the URLs from all the pages

In [2]:
def anyOffersOnTheList():
    try:
        driver.find_element(By.CSS_SELECTOR, '#main-offers-listing > div.hfenof > div.t2re51w > div')
        return False
    except:
        return True
    
offers_urls = []

def fetchOffersUrlsFromSinglePage():
    offersContainer = driver.find_element("xpath", '//*[@id="main-offers-listing"]/div[1]/div')
    offers = offersContainer.find_elements(By.CLASS_NAME, 'a4pzt2q ')
    # offers = offersContainer.find_elements(By.CSS_SELECTOR, '#offer-title')
    # print('\t'+ str(len(offers)) + ' offers:')
    for offer in offers:
        offers_urls.append(offer.get_property("href"))

page = 1
while True: # because not sure how many pages are there
    site = driver.get(base_url + "?pageNumber=" + str(page))
    if not anyOffersOnTheList():
        print('fetched ' + str(len(offers_urls)) + ' offer urls in total')
        break # break if no results
    else:
        time.sleep(random.uniform(0.5, 1)) #humanize
        fetchOffersUrlsFromSinglePage()
        print('page ' + str(page) + ' urls fetched')
        page += 1

page 1 urls fetched
page 2 urls fetched
page 3 urls fetched
page 4 urls fetched
page 5 urls fetched
page 6 urls fetched
page 7 urls fetched
page 8 urls fetched
page 9 urls fetched
page 10 urls fetched
page 11 urls fetched
page 12 urls fetched
fetched 572 offer urls in total


### Analyse offer

In [31]:
def offerNotFound():
    try:
        driver.find_element("xpath", '//*[@data-test="text-offerNotFound"]')
        return True
    except:
        return False
    
def getOfferDetails():
    #SALARY
    salaryContainer = driver.find_element(By.XPATH, '//*[@data-test="section-contract"]') # this element should always exist
    salary = salaryContainer.text
    # print(salary  + '\n')

    # EMPLOYER
    employerElement = driver.find_element("xpath", '//*[@data-test="anchor-company-link"]') # this element should always exist
    employer = employerElement.text + ' ' + employerElement.get_property("href")
    # print(employer  + '\n')
    
    #WORKFROM, EXP, VALIDTO, LOCATION - "PARAMETERS"
    workModes, positionLevels, offerValidTo, location = '', '', '', ''
    parametersContainer = driver.find_element(By.CLASS_NAME, "c21kfgf")
    parameters = parametersContainer.find_elements(By.CLASS_NAME, "s1bu9jax")
    for param in parameters:
        paramType = param.get_attribute("data-test") #element description
        match paramType:
            case "section-workModes":
                workModes = param.text
            case "section-positionLevels":
                positionLevels = param.text
            case "section-offerValidTo":
                offerValidTo = param.text
            case "section-workplace":
                location = param.text
                try: #to find and click 'more locations' button then fetch what's inside
                    moreLocations = driver.find_element("xpath", '//*[@data-test="button-locationPicker"]')
                    moreLocations.click()
                    # time.sleep(0.05) #probably necessary
                    locations = moreLocations.find_element("xpath", '//*[@data-test="modal-locations"]')
                    location = locations.text
                except:
                    pass #leave location as it was
    # print(workModes + '\n\n' + positionLevels + '\n\n' +  offerValidTo + '\n\n' +  location + '\n')

    # # # TECHSTACK
    descriptionsContainer = driver.find_element(By.CSS_SELECTOR, '#TECHNOLOGY_AND_POSITION')

    techstack = descriptionsContainer.find_elements(By.CLASS_NAME, "c1fj2x2p")
    techstackExpected = ''
    techstackOptional = ''
    for group in techstack:
        if group.text[0:8] == 'EXPECTED':
            techstackExpected = group.text[9:-1]
        elif group.text[0:8] == 'OPTIONAL':
            techstackOptional = group.text[9:-1]
    # print(techstackExpected + '\n\n' + techstackOptional + '\n')

    #RESPONSIBILITIES
    responsibilities = None
    try:
        try:
            responsibilities = descriptionsContainer.find_element("xpath", '//*[@data-test="section-responsibilities"]/ul').text #/only ul elements
        except:
            responsibilities = descriptionsContainer.find_element("xpath", '//*[@data-test="section-responsibilities"]').text #/if it's a single entry
    except:
        pass #do nothing as the value is already None
        print('RESPONSIBILITIES:\n' + str(responsibilities) + '\n' + driver.current_url)

    #REQUIREMENTS
    requirements = None
    try:
        try:
            requirements = descriptionsContainer.find_element("xpath", '//*[@data-test="section-requirements"]/ul').text
        except:
            requirements = descriptionsContainer.find_element("xpath", '//*[@data-test="section-requirements"]').text #/if it's a single entry
    except:
        pass #do nothing as the value is already None
        print('REQUIREMENTS:\n' + str(requirements) + '\n' + driver.current_url)


    #OPTIONAL REQUIREMENTS
    optionalRequirements = None
    try:
        optionalRequirementsContainer = descriptionsContainer.find_elements("xpath", '//*[@data-test="section-requirements-optional"]/li')
        if len(optionalRequirementsContainer) > 0:
            optionalRequirements = ''
            for optionalRequirement in optionalRequirementsContainer:
                optionalRequirements += optionalRequirement.text + '\n'
        elif len(optionalRequirementsContainer) <= 0:
            try:
                optionalRequirements = descriptionsContainer.find_element("xpath", '//*[@data-test="section-requirements-optional"]').text
            except:
                pass #do nothing as the value is already None
                # print('OPTIONAL:\n' + str(optionalRequirements) + '\n' + driver.current_url)        
    except:
        pass #do nothing as the value is already None
    # print('OPTIONAL:\n' + str(optionalRequirements) + '\n' + driver.current_url)

for i in range (6,7):
    driver.get(offers_urls[i])
    if not offerNotFound(): 
        getOfferDetails()
        print (str(i) + '/' + str(len(offers_urls)) + ' done')
    else:
        print('OFFER NOT FOUND: ' +  driver.current_url)
    # time.sleep(random.uniform(5,15)) #Humanize request times
# print('all doned')

remote work • hybrid work

senior

valid for 11 days
(from 19 Aug)

Greater Poland
Kuyavia-Pomerania
Lesser Poland
Łódź
Lower Silesia
Lublin
Lubusz
Masovian
Opole
Podlaskie
Pomeranian
Silesian
Subcarpathia
Świętokrzyskie
Warmian-Mazurian
West Pomeranian

6/572 done
